In [18]:
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [19]:
from flask import Flask, request, render_template, jsonify, json
from decouple import config
from flask_cors import CORS
import numpy as np
import networkx as nx
import tweepy
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import re
from collections import Counter
import asyncio
from concurrent.futures import ThreadPoolExecutor
import requests
import nest_asyncio
nest_asyncio.apply()

#set up twitter environment
TWITTER_AUTH =tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
TWITTER_AUTH.set_access_token(config('TWITTER_ACCESS_TOKEN'),config('TWITTER_ACCESS_TOKEN_SECRET'))
TWITTER = tweepy.API(TWITTER_AUTH)

In [20]:
def get_user_timeline(session, search, index, page):
    timeline = TWITTER.user_timeline(
        id=search,
        count=200,
        exclude_replies=False,
        include_rts=True,
        tweet_mode="extended",
        page=page,
    )
    index.append(
        [
            user_mentions["screen_name"]
            for um_list in [tweet.entities["user_mentions"] for tweet in timeline]
            for user_mentions in um_list
        ]
    )

def get_user_favorites(session, search, index, page):
    timeline = TWITTER.favorites(
        id=search,
        count=200,
        exclude_replies=False,
        include_rts=True,
        tweet_mode="extended",
        page=page,
    )
    index.append([tweet.author.screen_name for tweet in timeline])

In [25]:
nest_asyncio.apply()
import asyncio
import requests
from concurrent.futures import ThreadPoolExecutor


async def get_input_user_data_asynchronous(search, index):
    with ThreadPoolExecutor(max_workers=20) as executor:
        with requests.Session() as session:
            loop = asyncio.get_event_loop()
            tasks = [
                loop.run_in_executor(
                    executor, get_user_timeline, *(session, search, index, page)
                )
                for page in range(10)
            ] + [
                loop.run_in_executor(
                    executor, get_user_favorites, *(session, search, index, page)
                )
                for page in range(10)
            ]
            for response in await asyncio.gather(*tasks):
                pass



def execute_async_input_user_event_loop(search, index):
    """Really couldn't think of a proper name. 
    This function does something analogous to compiling the get_data_asynchronously funciont,
    Then it executre loop."""
    future = asyncio.ensure_future(get_input_user_data_asynchronous(search, index))
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)
    index = [i for sublist in index for i in sublist]
    top_interactions = [ i[0] for i in Counter(index).most_common(10) ]
    return top_interactions

In [26]:
index = []

In [27]:
%%time
z = execute_async_input_user_event_loop("austen",index)

Wall time: 1.34 s


In [28]:
z

['LambdaSchool',
 'paulg',
 'ne0liberal',
 'tommycollison',
 'jdcivicscience',
 'webdevMason',
 'patio11',
 'shaunmmaguire',
 'mckaywrigley',
 'TrevMcKendrick',
 'Austen',
 'rabois',
 'bjc290',
 'thogge',
 'pt',
 'justGLew',
 'calebhicks',
 'shl',
 'toomuchpete',
 'hazem_awad',
 'sbmiller5',
 'lpolovets',
 'rrhoover',
 'ryanallred',
 'MandaLane_',
 'SureReno',
 'typesfast',
 'danielgross',
 'tylercowen',
 'kimeejohnson',
 'misterohno',
 'JLHenriod',
 'tylerwillis',
 'AdamSinger',
 'kylebrussell',
 'sugabelly',
 'StevenGlinert',
 'collision',
 'rogerdickey',
 'asteroid_saku',
 'ccchaircut',
 'wawacat2',
 'elonmusk',
 'TheAnnaGat',
 'doug__h',
 'KarenDZachary',
 'ItsladyKadie',
 'jwdanner',
 'micsolana',
 'zackkanter']

In [40]:
async def get_index_data_asynchronous(search_list, index):
    with ThreadPoolExecutor(max_workers=20) as executor:
        with requests.Session() as session:
            loop = asyncio.get_event_loop()
            tasks = [
                loop.run_in_executor(
                    executor, get_user_timeline, *(session, search, index, 1)
                )
                for search in search_list
            ] + [
                loop.run_in_executor(
                    executor, get_user_favorites, *(session, search, index, 1)
                )
               for search in search_list
            ]
            for response in await asyncio.gather(*tasks):
                pass
            
def execute_async_index_event_loop(search_list, index):
    """Really couldn't think of a proper name. 
    This function does something analogous to compiling the get_data_asynchronously funciont,
    Then it executre loop."""
    future = asyncio.ensure_future(get_index_data_asynchronous(search_list, index))
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)
    #index = [i for sublist in index for i in sublist]
    #top_interactions = [ i[0] for i in Counter(index).most_common(5)
    return index

In [41]:
index = []

In [42]:
%%time
f = execute_async_index_event_loop(z,index)

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [43]:
len(f)

35258